# <center> **Requesting an external API**

A partir d'une liste de titres de films nous allons requêter l'API publique [https://www.omdbapi.com](https://www.omdbapi.com)

Nous enregistrerons les données non-structurées (résumé et affiche du film) dans une base NoSQL (MongoDB)

In [83]:
%reset

## **Imports**

In [84]:
import math
import copy
import re
import json
import requests
import numpy as np
import pandas as pd
from tqdm import tqdm
from unidecode import unidecode

# MongoDB / Pymongo
import pymongo
from pymongo.mongo_client import MongoClient
from pymongo.server_api import ServerApi
import pprint

pd.set_option('display.max_rows', 10)
tqdm.pandas()

api_key = "b8dd5759"

## **Reading the data**


In [ ]:
df_movies = pd.read_csv('csv/movies_year_2000_to_2003.csv', delimiter = ',', usecols=['title', 'original_title', 'summary', 'url_thumbnail'])
print("Nb movies :", df_movies.shape[0])
df_movies.head(5)

Nb movies : 807


,title,original_title,summary,url_thumbnail
0,Une histoire vraie,The Straight Story,"Alvin Straight, vétéran de 73 ans, vit avec sa...",https://fr.web.img6.acsta.net/c_310_420/pictur...
1,La Ligne verte,The Green Mile,"Paul Edgecomb, pensionnaire centenaire d'une m...",https://fr.web.img2.acsta.net/c_310_420/medias...
2,Matrix,The Matrix,Programmeur anonyme dans un service administra...,https://fr.web.img4.acsta.net/c_310_420/medias...
3,Eyes Wide Shut,Eyes Wide Shut,"William Harford, médecin, mène une paisible ex...",https://fr.web.img6.acsta.net/c_310_420/medias...
4,Fight Club,Fight Club,"Le narrateur, sans identité précise, vit seul,...",https://fr.web.img6.acsta.net/c_310_420/pictur...
...,...,...,...,...
802,Un vampire à Brooklyn,Vampire in Brooklyn,Maximillian appartient à une longue lignée de ...,https://fr.web.img6.acsta.net/c_310_420/medias...
803,Fair Game,Fair Game,Pour avoir menacé son ex-mari de faire saisir ...,https://fr.web.img2.acsta.net/c_310_420/medias...
804,Les Maitres du monde,The Puppet Masters,"Sam Nivens, jeune agent du gouvernement, est e...",https://fr.web.img2.acsta.net/c_310_420/medias...
805,Halloween 6 : La Malédiction de Michael Myers,Halloween: The Curse of Michael Myers,Dix ans après avoir terrorisé la petite ville ...,https://fr.web.img4.acsta.net/c_310_420/medias...


In [166]:
def format_string(st):
    ''' format string 
        from "title of the movie" 
        to title+of+the+movie

        Arg: st string to be converted.
    '''
    res = ''
    for c in st:
        if c.isdigit() or c.isalpha() or c.isspace():
            res += unidecode(c)
        else:
            res += ' '
    return '+'.join([word for word in res.split() if len(word) > 1])

def request_omdb_from_title(title):
    ''' Request the omdb API
    
        return a json dictionary with the information about the movie.

        Arg:
         - title: string with title of the movie we want the infos about.
    '''
    url = f"https://www.omdbapi.com/?apikey={api_key}&t={format_string(title)}"
    r = requests.get(url)
    if r.status_code != 200:
        print(f"ERROR {title}, Response Code: {r.status_code}")
        print("Request:", url)
        return {'Response': 'False'}
    return json.loads(r.text)

def get_plot_and_thumbail_from_omdb(title):
    ''' return movie plot and thumbail through an API request.
        
        return: 
          - plot:      string containing the plot of the movie,
          - thumbnail: string containing the url of the thumbnail.

        Arg: title: string with the title of the movie.
    '''
    plot, thumbnail = '', ''
    res_dict = request_omdb_from_title(title)
    lst_keys = res_dict.keys()
    assert 'Response' in res_dict
    if res_dict['Response'] == 'True':
        # print('res_dict', res_dict)
        assert 'Plot' in lst_keys and 'Poster' in lst_keys
        if res_dict['Plot'] != 'N/A':
            plot = res_dict['Plot']
        if res_dict['Poster'] != '' and res_dict['Poster'] != 'N/A':
            thumbnail = res_dict['Poster']
    return plot + "AND" + thumbnail

## **Get the plot and the thumbnail from the omdb API**

In [167]:
df_movies['temp']      = df_movies['original_title'].apply(get_plot_and_thumbail_from_omdb)
df_movies['plot']      = df_movies['temp'].apply(lambda x : x.split('AND')[0])
df_movies['thumbnail'] = df_movies['temp'].apply(lambda x : x.split('AND')[1])
df_movies['plot']          = np.where(df_movies['plot'] != '', df_movies['plot'], df_movies['summary'])
df_movies['url_thumbnail'] = np.where(df_movies['thumbnail'] != '', df_movies['thumbnail'], df_movies['url_thumbnail'])
df_movies = df_movies[['title', 'original_title', 'plot', 'url_thumbnail']]

## **Store the data in a NoSQL database**

In [202]:
# Connect to MongoDB
client = pymongo.MongoClient("mongodb://localhost:27017/")

# Create database "allocine" (or selects it if already exists)
mydb = client["allocine"]
# client.drop_database("movies")

# Create a collection "movies" (table in SQL)
col_movies = mydb["movies"]
# col_movies.drop()

In [ ]:
# Insertion of movie plots in MongoDB database
col_movies.insert_many(df_movies.to_dict(orient='records')) # TO DO ONLY ONCE
# col_movies.drop()

print(client.list_database_names())
print(mydb.list_collection_names())

# for doc in list(col_movies.find().limit(5)):
#     pprint.pprint(doc)

print("Nb documents:", col_movies.count_documents({}))

['Rennes', 'Rennes2', 'admin', 'allocine', 'config', 'local', 'mydatabase', 'test_queries']
['movies']
Nb documents: 3440


## **Checking what is in the Mongo DB**

In [ ]:
df1 = pd.read_csv('csv/movies_year_1960_to_1970.csv', delimiter = ',', usecols=['title', 'original_title', 'summary', 'url_thumbnail'])
df2 = pd.read_csv('csv/movies_year_1970_to_1980.csv', delimiter = ',', usecols=['title', 'original_title', 'summary', 'url_thumbnail'])
df3 = pd.read_csv('csv/movies_year_1980_to_1990.csv', delimiter = ',', usecols=['title', 'original_title', 'summary', 'url_thumbnail'])
df4 = pd.read_csv('csv/movies_year_1990_to_1995.csv', delimiter = ',', usecols=['title', 'original_title', 'summary', 'url_thumbnail'])
df5 = pd.read_csv('csv/movies_year_1995_to_2000.csv', delimiter = ',', usecols=['title', 'original_title', 'summary', 'url_thumbnail'])
df6 = pd.read_csv('csv/movies_year_2000_to_2003.csv', delimiter = ',', usecols=['title', 'original_title', 'summary', 'url_thumbnail'])
df7 = pd.read_csv('csv/movies_year_2003_to_2006.csv', delimiter = ',', usecols=['title', 'original_title', 'summary', 'url_thumbnail'])
df8 = pd.read_csv('csv/movies_year_2006_to_2008.csv', delimiter = ',', usecols=['title', 'original_title', 'summary', 'url_thumbnail'])
df9 = pd.read_csv('csv/movies_year_2008_to_2010.csv', delimiter = ',', usecols=['title', 'original_title', 'summary', 'url_thumbnail'])
df10 = pd.read_csv('csv/movies_year_2010.csv', delimiter = ',', usecols=['title', 'original_title', 'summary', 'url_thumbnail'])

lst = [df1, df2, df3, df4, df5, df6, df7, df8, df9, df10]
df_csv = pd.concat(lst)
# print(df_csv.shape[0])

for df in lst:
    print(df.shape[0])

# Reading data fmo Mongo DB
client = pymongo.MongoClient("mongodb://localhost:27017/")
mydb = client["allocine"]
col_movies = mydb["movies"]
print("Nb documents:", col_movies.count_documents({}))

lst = list(col_movies.find())
df_mongo = pd.DataFrame(lst, columns = ['_id', 'title', 'original_title', 'plot', 'url_thumbnail'])
print(df_mongo.shape[0])

# 1960 to 2000 stored in Mongo DB
518 + 678 + 869 + 585 + 807 # ~ 3440

518
678
869
585
807
729
980
863
983
498
Nb documents: 3440
3440


3457

In [210]:
df4

,title,original_title,summary,url_thumbnail
0,Forrest Gump,Forrest Gump,"Quelques décennies d'histoire américaine, des ...",https://fr.web.img2.acsta.net/c_310_420/pictur...
1,Pulp Fiction,Pulp Fiction,L'odyssée sanglante et burlesque de petits mal...,https://fr.web.img2.acsta.net/c_310_420/medias...
2,Les Evadés,The Shawshank Redemption,"Red, condamné à perpétuité, et Andy Dufresne, ...",https://fr.web.img6.acsta.net/c_310_420/medias...
3,Léon,Léon,Un tueur à gages répondant au nom de Léon pren...,https://fr.web.img6.acsta.net/c_310_420/pictur...
4,Entretien avec un vampire,Interview with the Vampire,San Francisco dans les années 90. Un jeune jou...,https://fr.web.img2.acsta.net/c_310_420/medias...
...,...,...,...,...
580,Frankenhooker,Frankenhooker,"Jeffrey Kranken, scientifique fou, a la douleu...",https://fr.web.img6.acsta.net/c_310_420/medias...
581,Moon 44,Moon 44,"En 2038, les ressources naturelles de la Terre...",https://fr.web.img6.acsta.net/c_310_420/medias...
582,Maniac Cop 2,Maniac Cop 2,L'officier Matt Cordell n'est pas mort. Cette ...,https://fr.web.img2.acsta.net/c_310_420/medias...
583,Dragon Ball Z : Le Combat fratricide,Doragon bôru Z 3: Chikyû marugoto chô kessen,"Un nouveau Saïyen, Thalès, qui ressemble à s'y...",https://fr.web.img6.acsta.net/c_310_420/medias...


In [207]:
# Store data in csv file
df_mongo.to_csv('csv/test.csv', sep=',', index = False)

**Since we added twice the same 807 movies, we have to remove them from MongoDB**

In [ ]:
client = pymongo.MongoClient("mongodb://localhost:27017/")
mydb = client["allocine"]
col_movies = mydb["movies"]

print("Nb documents:", col_movies.count_documents({}))
col_movies.delete_many({'original_title' : {'$in' : df_movies['original_title'].values.tolist()} })
print("Nb documents:", col_movies.count_documents({}))

Nb documents: 4264
Nb documents: 2639


**Looks like more documents have been deleted**<br>
So we will drop the whole data base and use the csv 'mongoDB.csv' to restore the previous database and re-add the movies needed.

In [ ]:
client = pymongo.MongoClient("mongodb://localhost:27017/")
client.drop_database('allocine')

# Recreate DB
client = pymongo.MongoClient("mongodb://localhost:27017/")
mydb = client["allocine"]
col_movies = mydb["movies"]

## **Query MongoDB Using PyMongo**

### **We create another DB**

In [184]:
client2 = pymongo.MongoClient("mongodb://localhost:27017/")
mydb2 = client2["test_queries"]
# client.drop_database("test_queries")

# Create a collection "movies" (table in SQL)
col_fruits = mydb2["fruits"]
# col_fruits.drop()

col_fruits.insert_many([
        # { "_id": 1, "name": "apples", "qty": 5, "rating": 3, "color": "red", "type": ["fuji", "honeycrisp"] },
        # { "_id": 2, "name": "bananas", "qty": 7, "rating": 4, "color": "yellow", "type": ["cavendish"] },
        { "_id": 3, "name": "oranges", "qty": 6, "rating": 2, "type": ["naval", "mandarin"] },
        { "_id": 4, "name": "pineapple", "qty": 3, "rating": 5, "color": "yellow" },
        { "_id": 5, "name": "test", "qty": 10, "rating": 3, "color": "green", "type": ["fuji", "typeXXX"] },
    ])
# col_fruits.drop()

print(client2.list_database_names())
print(mydb2.list_collection_names())

['Rennes', 'Rennes2', 'admin', 'allocine', 'config', 'local', 'mydatabase', 'test_queries']
['fruits']


**Sources**<br>
https://www.mongodb.com/docs/languages/python/pymongo-driver/current/read/specify-a-query/<br>
https://www.w3resource.com/mongodb/introduction-mongodb.php<br>
https://www.mongodbtutorial.org/mongodb-crud/mongodb-findone/<br>
https://geekflare.com/fr/mongodb-queries-examples/<br>


**Exact Match**

In [175]:
results = col_movies.find({ "title": "Eyes Wide Shut" })
results.to_list()

[{'_id': ObjectId('67b1ae3a1e1d959e842460ed'),
  'title': 'Eyes Wide Shut',
  'original_title': 'Eyes Wide Shut',
  'plot': "A Manhattan doctor embarks on a bizarre, night-long odyssey after his wife's admission of unfulfilled longing.",
  'url_thumbnail': 'https://m.media-amazon.com/images/M/MV5BZTQ0MmM5MDAtYmYyZS00MzlmLTlhZTAtZDJlZWY5ZTZkZjZmXkEyXkFqcGc@._V1_SX300.jpg'},
 {'_id': ObjectId('67b1ae4a1e1d959e84246414'),
  'title': 'Eyes Wide Shut',
  'original_title': 'Eyes Wide Shut',
  'plot': "A Manhattan doctor embarks on a bizarre, night-long odyssey after his wife's admission of unfulfilled longing.",
  'url_thumbnail': 'https://m.media-amazon.com/images/M/MV5BZTQ0MmM5MDAtYmYyZS00MzlmLTlhZTAtZDJlZWY5ZTZkZjZmXkEyXkFqcGc@._V1_SX300.jpg'}]

In [144]:
results = col_fruits.find({ "color": "yellow" })
results.to_list()

[{'_id': 2,
  'name': 'bananas',
  'qty': 7,
  'rating': 4,
  'color': 'yellow',
  'type': ['cavendish']},
 {'_id': 4, 'name': 'pineapple', 'qty': 3, 'rating': 5, 'color': 'yellow'}]

**Comparison operators**

list of operators: https://www.mongodb.com/docs/manual/reference/operator/query-comparison/

In [143]:
results = col_fruits.find({ "rating": { "$gt" : 2 }})
for f in results:
    print(f) 

{'_id': 1, 'name': 'apples', 'qty': 5, 'rating': 3, 'color': 'red', 'type': ['fuji', 'honeycrisp']}
{'_id': 2, 'name': 'bananas', 'qty': 7, 'rating': 4, 'color': 'yellow', 'type': ['cavendish']}
{'_id': 4, 'name': 'pineapple', 'qty': 3, 'rating': 5, 'color': 'yellow'}


**Logical operators**

https://www.mongodb.com/docs/manual/reference/operator/query-logical/

In [147]:
results = col_fruits.find({ 
    "$or": [
        { "qty": { "$gt": 5 }},
        { "color": "yellow" }
    ]
})
for f in results:
    print(f)

{'_id': 2, 'name': 'bananas', 'qty': 7, 'rating': 4, 'color': 'yellow', 'type': ['cavendish']}
{'_id': 3, 'name': 'oranges', 'qty': 6, 'rating': 2, 'type': ['naval', 'mandarin']}
{'_id': 4, 'name': 'pineapple', 'qty': 3, 'rating': 5, 'color': 'yellow'}


**Arrays operators**

https://www.mongodb.com/docs/manual/reference/operator/query-array/

In [158]:
for f in col_fruits.find({}):
    print(f)

print('----')
results = col_fruits.find({
    "type" : { "$size": 2 }
})

for f in results:
    print(f)

{'_id': 1, 'name': 'apples', 'qty': 5, 'rating': 3, 'color': 'red', 'type': ['fuji', 'honeycrisp']}
{'_id': 2, 'name': 'bananas', 'qty': 7, 'rating': 4, 'color': 'yellow', 'type': ['cavendish']}
{'_id': 3, 'name': 'oranges', 'qty': 6, 'rating': 2, 'type': ['naval', 'mandarin']}
{'_id': 4, 'name': 'pineapple', 'qty': 3, 'rating': 5, 'color': 'yellow'}
{'_id': 5, 'name': 'test', 'qty': 10, 'rating': 3, 'color': 'green', 'type': ['fuji', 'typeXXX']}
----
{'_id': 1, 'name': 'apples', 'qty': 5, 'rating': 3, 'color': 'red', 'type': ['fuji', 'honeycrisp']}
{'_id': 3, 'name': 'oranges', 'qty': 6, 'rating': 2, 'type': ['naval', 'mandarin']}
{'_id': 5, 'name': 'test', 'qty': 10, 'rating': 3, 'color': 'green', 'type': ['fuji', 'typeXXX']}


In [159]:
results = col_fruits.find({
    "type" : { "$all": ['fuji'] }
})

for f in results:
    print(f)

{'_id': 1, 'name': 'apples', 'qty': 5, 'rating': 3, 'color': 'red', 'type': ['fuji', 'honeycrisp']}
{'_id': 5, 'name': 'test', 'qty': 10, 'rating': 3, 'color': 'green', 'type': ['fuji', 'typeXXX']}


**Element Operator**

to check if a field exists

In [160]:
results = col_fruits.find( { "color" : { "$exists": "true" }} )
for f in results:
    print(f)

{'_id': 1, 'name': 'apples', 'qty': 5, 'rating': 3, 'color': 'red', 'type': ['fuji', 'honeycrisp']}
{'_id': 2, 'name': 'bananas', 'qty': 7, 'rating': 4, 'color': 'yellow', 'type': ['cavendish']}
{'_id': 4, 'name': 'pineapple', 'qty': 3, 'rating': 5, 'color': 'yellow'}
{'_id': 5, 'name': 'test', 'qty': 10, 'rating': 3, 'color': 'green', 'type': ['fuji', 'typeXXX']}


**Evaluation operators**

https://www.mongodb.com/docs/manual/reference/operator/query-evaluation/

In [ ]:
results = col_fruits.find({ "name" : { "$regex" : "p{2,}" }} ) ## At least 2 consecutives p
for f in results:
    print(f)

{'_id': 1, 'name': 'apples', 'qty': 5, 'rating': 3, 'color': 'red', 'type': ['fuji', 'honeycrisp']}
{'_id': 4, 'name': 'pineapple', 'qty': 3, 'rating': 5, 'color': 'yellow'}


**Delete data**

In [ ]:
# Delete_one
for f in col_fruits.find({}):
    print(f)

col_fruits.delete_one({'_id' : 4})

for f in col_fruits.find({}):
    print(f)

{'_id': 1, 'name': 'apples', 'qty': 5, 'rating': 3, 'color': 'red', 'type': ['fuji', 'honeycrisp']}
{'_id': 2, 'name': 'bananas', 'qty': 7, 'rating': 4, 'color': 'yellow', 'type': ['cavendish']}
{'_id': 3, 'name': 'oranges', 'qty': 6, 'rating': 2, 'type': ['naval', 'mandarin']}
{'_id': 5, 'name': 'test', 'qty': 10, 'rating': 3, 'color': 'green', 'type': ['fuji', 'typeXXX']}
{'_id': 4, 'name': 'pineapple', 'qty': 3, 'rating': 5, 'color': 'yellow'}
{'_id': 1, 'name': 'apples', 'qty': 5, 'rating': 3, 'color': 'red', 'type': ['fuji', 'honeycrisp']}
{'_id': 2, 'name': 'bananas', 'qty': 7, 'rating': 4, 'color': 'yellow', 'type': ['cavendish']}
{'_id': 3, 'name': 'oranges', 'qty': 6, 'rating': 2, 'type': ['naval', 'mandarin']}
{'_id': 5, 'name': 'test', 'qty': 10, 'rating': 3, 'color': 'green', 'type': ['fuji', 'typeXXX']}
{'_id': 4, 'name': 'pineapple', 'qty': 3, 'rating': 5, 'color': 'yellow'}


In [ ]:
# Delete_many
for f in col_fruits.find({}):
    print(f)

col_fruits.delete_many({'_id' : {'$gte' : 3} })

for f in col_fruits.find({}):
    print(f)

{'_id': 1, 'name': 'apples', 'qty': 5, 'rating': 3, 'color': 'red', 'type': ['fuji', 'honeycrisp']}
{'_id': 2, 'name': 'bananas', 'qty': 7, 'rating': 4, 'color': 'yellow', 'type': ['cavendish']}
{'_id': 3, 'name': 'oranges', 'qty': 6, 'rating': 2, 'type': ['naval', 'mandarin']}
{'_id': 4, 'name': 'pineapple', 'qty': 3, 'rating': 5, 'color': 'yellow'}
{'_id': 5, 'name': 'test', 'qty': 10, 'rating': 3, 'color': 'green', 'type': ['fuji', 'typeXXX']}
{'_id': 1, 'name': 'apples', 'qty': 5, 'rating': 3, 'color': 'red', 'type': ['fuji', 'honeycrisp']}
{'_id': 2, 'name': 'bananas', 'qty': 7, 'rating': 4, 'color': 'yellow', 'type': ['cavendish']}
{'_id': 3, 'name': 'oranges', 'qty': 6, 'rating': 2, 'type': ['naval', 'mandarin']}
{'_id': 4, 'name': 'pineapple', 'qty': 3, 'rating': 5, 'color': 'yellow'}
{'_id': 5, 'name': 'test', 'qty': 10, 'rating': 3, 'color': 'green', 'type': ['fuji', 'typeXXX']}


In [186]:
# Delete_many
for f in col_fruits.find({}):
    print(f)

col_fruits.delete_many({'_id' : {'$in' : [1, 3, 5]} })

for f in col_fruits.find({}):
    print(f)

{'_id': 1, 'name': 'apples', 'qty': 5, 'rating': 3, 'color': 'red', 'type': ['fuji', 'honeycrisp']}
{'_id': 2, 'name': 'bananas', 'qty': 7, 'rating': 4, 'color': 'yellow', 'type': ['cavendish']}
{'_id': 3, 'name': 'oranges', 'qty': 6, 'rating': 2, 'type': ['naval', 'mandarin']}
{'_id': 4, 'name': 'pineapple', 'qty': 3, 'rating': 5, 'color': 'yellow'}
{'_id': 5, 'name': 'test', 'qty': 10, 'rating': 3, 'color': 'green', 'type': ['fuji', 'typeXXX']}
{'_id': 2, 'name': 'bananas', 'qty': 7, 'rating': 4, 'color': 'yellow', 'type': ['cavendish']}
{'_id': 4, 'name': 'pineapple', 'qty': 3, 'rating': 5, 'color': 'yellow'}
